# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импорт библиотеки pandas.
Вывод первых 10 строк таблицы для изучения данных.

In [1]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и сохранение в data
data = pd.read_csv('/datasets/data.csv')

# получение первых 10 строк таблицы data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# получение общей информации о данных в таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов.

Согласно документации к данным:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах `days_employed` и `total_income` равны 19351, но меньше общего количества строк таблицы 21525.

**Вывод**

В каждой строке таблицы — статистика о платёжеспособности клиентов.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, в столбцах стаж и доход. Уровень образования заполнен в разных регистрах. Так же в столбце стаж есть отрицательные числа, и завышеные.

Перед тем как ответить на вопросы, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем сколько в таблице пропущенных значений

In [3]:
# подсчет количества пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В итоге пропуски только в двух столбцах days_employed(общий трудовой стаж в днях) и total_income(ежемесячный доход). Заменим пустые значения на ноль и снова проверим количество пропущенных значений.

In [4]:
# проверка что доход и стаж незаполнены в одних строках
data[data['days_employed'].isna() == True]['total_income'].isna().sum()

2174

In [5]:
# проверка завышенных показателей
data[data['days_employed']/365 > 80]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

In [6]:
# проверка завышенных показателей
data[data['days_employed']/365 < 80]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

Проверка показала, что дни стажа по пенсионерам и безработным завышены. Эти данный будут искажать итоговый вывод. Надо уточнить данные у заказчика.

In [7]:
# меняем отрицательное значение стажа на положительное
def days_employed_abs(index):
    if index < 0:
        index *= -1
        return index
    else:
        return index
data['days_employed'] = data['days_employed'].apply(days_employed_abs)

# находим среднее значение по столбцу общий трудовой стаж в днях, без учета пенсионеров и безработных
# находим медиану по столбцу ежемесячный доход
days_employed_mean = data[data['days_employed']/365 < 80]['days_employed'].mean()
total_income_median = data['total_income'].median()

# заменим пустые строки
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data['total_income'] = data['total_income'].fillna(total_income_median)

# подсчет количества пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Замена пропущенных значений выполнена, что показывает повторный пересчет пропусков по таблице data. Возможная причина пустых ячеек клиент не указал данные. По стажу замена была выполнена на среднее значение, а так же выполнена замена отрицательных чисел на положительные. В доходе выполнена замена на медианное значение.
Есть в таблице данные по пенсионерам и безработным, которые сильно завышены.

### Замена типа данных

In [8]:
# замена вещественных данных на тип целочисленный
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

**Вывод**

В столбцах стаж и доход выполнена замена типа данных на целочисленное

### Обработка дубликатов

Поправим формат в столбце уровень образования, ранее было видно, что регистр разный. Так же проверим как заполнены другие столбцы

In [9]:
# в столбце уровень образования приведем к нижнему регистру
data['education'] = data['education'].str.lower()
# проверка формата уровень образования клиента после привидения к нижнему регистру
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [10]:
# проверка формата
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [11]:
# проверка формата семейное положение
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
# проверка формата тип занятости
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [13]:
# проверка формата цель получения кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [14]:
# проверка формата количество детей в семье
data['children'].value_counts()

# в таблице присутствуют данные с отрицательным значением по количеству детей в семье и 20 детей
# заменим отрицательное значение -1 на 1, а 20 на медианое значение,т.к. не ясно как возникла ошибка
data['children'] = data['children'].replace(-1, 1)
children_median = data[data['children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)
display(data['children'].value_counts())

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [15]:
# методом duplicated() находим строки-дубликаты, у которых полностью совпадают значения в ячейках
data.duplicated().sum()
# явных вышло 71 строка
# удаляем явные дубликаты метод drop_duplicates()
data = data.drop_duplicates().reset_index(drop=True)

**Вывод**

Выявлены явные дубликаты в таблице data, применены методы для их удаления.
В столбце образование были выявлены дубликаты из за разного регистра, заменены на нижний регистр. Провели проверку форматов, привели к одному типу с помощью метода str.lower(). Проверили столбцы методом unique() и value_counts(). В столбце пол клиента обнаружено значение 'XNA', из данных не получится выяснить настоящий пол клиента, можно оставить так как пол не учувствует в расчетах. Также скорректирован столбец количество детей в семье, исправлены отрицательные значения и завышенное 20 детей. Завышенное заменено на медианное значение.

### Лемматизация

Раннее при обработке целей кредита было видно, что очень похожи. Для примера "покупка жилья", "операции с жильем", "покупка недвижимости" или "жилье" все это связано с недвижимостью, поэтому их можно объеденить одной категорией "недвижимость". Таким же способом мы объеденим кредиты на свадьбу, на покупку автомобиля и получение образования.

In [ ]:
# Проведем лемматизацию целей кредита
from pymystem3 import Mystem
m = Mystem() 

data['purpose_lem'] = 0

# фунция, где создаем новый столбец, объединяем в общие цели
def lemmatize_data(table):
    for line in range(0, len(table['purpose'])):
        lemmas = m.lemmatize(table.loc[line, 'purpose'])
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            table.loc[line, 'purpose_lem'] = 'недвижимость'
        if 'свадьба' in lemmas:
            table.loc[line, 'purpose_lem'] = 'свадьба'
        if 'автомобиль' in lemmas:
            table.loc[line, 'purpose_lem'] = 'автомобиль'
        if 'образование' in lemmas:
            table.loc[line, 'purpose_lem'] = 'образование'
    return table

data = lemmatize_data(data)
# проверка что нет незаполненых строк
data['purpose_lem'].value_counts()

In [ ]:
purposes_keys = {'жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба'}

def get_purpose(data):
    
    """Присваивает строке категорию цели"""
    
    intersection = list(purposes_keys & set(m.lemmatize(data['purpose'])))
    
    if not intersection:
        return 'категория не определена'
    return intersection[0]

df_example = data.head().copy()
df_example['new_purpose'] = df_example.apply(get_purpose, axis=1)

df_example.apply(get_purpose, axis=1)

**Вывод**

При проверке формата, было выявлено, что в основном цели кредита можно привести к 4 основным категориям: недвижимость, автомобиль, образование и свадьба. С помощью лемматизации разбили строки на основные категории. В дальнейшем упрости работу для изучения влияния цели кредита на другие метрики.

### Категоризация данных

Выполним катигоризацию по ежемесечному доходу. В дальнешем нам понадобиться это для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?".

In [ ]:
# выполним категоризацию и сразу добавим в отдельный столбце
data['income_id_category'] = pd.qcut(data['total_income'], 10, labels=False)

**Вывод**

Разделили уровень дохода на 10 категорий и добавили в новый столбце в таблицу.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# для расчета зависмости создаем новый объект
children_debt = pd.DataFrame()
# посчитаем количество строк по количеству детей
children_debt['debt_count'] = data.groupby('children')['debt'].count()
# посчитаем сумму строк с задолженостями
children_debt['debt_sum'] = data.groupby('children')['debt'].sum()
# посчитаем конверсию
children_debt['debt_ratio'] = children_debt['debt_sum'] / children_debt['debt_count']
children_debt.sort_values('debt_ratio', ascending = False)

In [ ]:
children_debt.style.format({'debt_ratio': '{:.2%}'})

**Вывод**

В таблице видно, бездетные чаще платят в срок. Если у клиента 4 детей количество просроченных платежей выше, но так же видно, что у тех кого 3 детей реже просрачивают чем люди с 1 ребенком. По даным сложно судить зависит наличие детей на возврат кредита в срок или нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# для расчета зависмости создаем новый объект
family_status_debt = pd.DataFrame()
# посчитаем количество строк в зависимости от семейного положения
family_status_debt['debt_count'] = data.groupby('family_status')['debt'].count()
# посчитаем сумму строк с задолженостями в зависимости от семейного положения
family_status_debt['debt_sum'] = data.groupby('family_status')['debt'].sum()
# посчитаем конверсию
family_status_debt['debt_ratio'] = family_status_debt['debt_sum'] / family_status_debt['debt_count'] 
family_status_debt.sort_values('debt_ratio', ascending = False)

In [ ]:
data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean'])

**Вывод**

Конверсия показала, реже задолженность имеют клиенты в разводе или овдовевшие. А люди не бывавшие в браке наоборот чаще имеют задолженность по возврату кредитов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# для расчета зависимости категоризируем уровень дохода, больше 1 млн. поставим категорию 10
# категории поделим по 100 тыс.
# def income_category(index):
#    if index >= 1000000:
#        return 10
#    else:
#        index = index // 100000
#        return index

# применим функцию и вставим в новый столбце
# data['income_id_category'] = data['total_income'].apply(income_category)
# для расчета зависмости создаем новый объект
total_income_debt = pd.DataFrame()
# посчитаем количество строк в зависимости от уровня дохода
total_income_debt['debt_count'] = data.groupby('income_id_category')['debt'].count()
# посчитаем сумму строк с задолженостями в зависимости от уровня дохода
total_income_debt['debt_sum'] = data.groupby('income_id_category')['debt'].sum()
# посчитаем конверсию
total_income_debt['debt_ratio'] = total_income_debt['debt_sum'] / total_income_debt['debt_count']
total_income_debt.sort_values('debt_ratio', ascending = False)

**Вывод**

Как показывают данные в таблице, зависимости между уровнем дохода и возвратом кредита в срок нет.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# для расчета зависмости создаем новый объект
purpose_category_debt = pd.DataFrame()
# посчитаем количество строк в зависимости от цели кредита
purpose_category_debt['debt_count'] = data.groupby('purpose_lem')['debt'].count()
# посчитаем сумму строк с задолженостями в зависимости от цели кредита
purpose_category_debt['debt_sum'] = data.groupby('purpose_lem')['debt'].sum()
# посчитаем конверсию
purpose_category_debt['debt_ratio'] = purpose_category_debt['debt_sum'] / purpose_category_debt['debt_count'] 
purpose_category_debt.sort_values('debt_ratio', ascending = False)

**Вывод**

Видим, что реже просрачивают клиенты которые берут кредит на недвижемость и свадьбу. А вот на автомобиль и образование чаще всего просрочка по кредиту.

## Шаг 4. Общий вывод

Для проверки гипотез выполнена работа по удалению пропусков, корректировка разных регистров, замене отрицательных и завышеных значений.  Проведена лемматизация и категоризация некоторых столбцов для ответа на вопросы нашей гипотизы.

В итоге после проведения анализа, выяснилось, что у бездетных клиентов реже просрочка по кредиту. Чем больше детей, тем чаще не платят в срок. Реже просрочивают люди берущие кредит на недвижимость и свадьбу. Так же оказалось, что те кто были в браке платят в срок, чем те которые не были.

Выходит что идеальный клиент который будет платить вовремя это - бездетный, в браке или был в браке, с целью провести свадьбу или купить недвижимость.